In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101, load_proxylessnas, load_nasbench_nlp
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor
from input_preprocessing import preprocess_nasbench, preprocess_nasbench_nlp, preprocess_proxylessnas

from scipy.stats import kendalltau as tau
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

### NAS-Bench-101

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 41 seconds


In [4]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [85]:
models = get_N_samples(119+100000)

In [86]:
reg = regressor([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]], models['val_acc'][:79])

In [87]:
y_pred = reg.predict([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]])
y_true = models['val_acc'][:79]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.5764910746410353 0.7550668670711767 0.963155304889156


In [88]:
np.save('outputs/nasbench_y_true_train.npy', y_true)
np.save('outputs/nasbench_y_pred_train.npy', y_pred)
np.save('outputs/nasbench_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_train.npy', r2_score(y_true, y_pred))

In [89]:
y_pred = reg.predict([models['X'][79:119], models['norm_A'][79:119], models['norm_AT'][79:119]])
y_true = models['val_acc'][79:119]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

2.6771875509159644 0.6835067426013202 0.842393547930542


In [90]:
np.save('outputs/nasbench_y_true_val.npy', y_true)
np.save('outputs/nasbench_y_pred_val.npy', y_pred)
np.save('outputs/nasbench_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_val.npy', r2_score(y_true, y_pred))

In [91]:
y_pred = reg.predict([models['X'][119:], models['norm_A'][119:], models['norm_AT'][119:]])
y_true = models['val_acc'][119:]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

7.555077454905181 0.690390307123404 0.6433056494273304


In [92]:
min_idx = np.argmin(y_pred.ravel())
max_idx = np.argmax(y_pred.ravel())

In [93]:
y_idx = np.where(y_pred > y_pred[min_idx])[0]

In [94]:
y_pred_10 = np.vstack([y_pred[min_idx], y_pred[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_pred[max_idx]]).ravel()
y_true_10 = np.hstack([y_true[min_idx], y_true[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_true[max_idx]]).ravel()
print(mean_squared_error(y_true_10, y_pred_10), tau(y_true_10, y_pred_10).correlation, r2_score(y_true_10, y_pred_10))

5.691547673229212 0.8222222222222221 0.37857951012643676


In [95]:
np.save('outputs/nasbench_y_true_test.npy', y_true)
np.save('outputs/nasbench_y_pred_test.npy', y_pred)
np.save('outputs/nasbench_y_true_test_10.npy', y_true_10)
np.save('outputs/nasbench_y_pred_test_10.npy', y_pred_10)
np.save('outputs/nasbench_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_test.npy', r2_score(y_true, y_pred))

### ProxylessNAS

CIFAR-100

In [16]:
models, val_acc = [], []
model_acc = np.load(f'outputs/model_acc_cifar100.npy', allow_pickle=True)
for m in model_acc:
    models.append(m['arch'])
    val_acc.append(m['val_acc'] * 100.)

In [17]:
data = preprocess_proxylessnas(models)
val_acc = np.array(val_acc)
train_data = [data['X'][:76], data['norm_A'][:76], data['norm_AT'][:76]]
valid_data = [data['X'][76:76+5], data['norm_A'][76:76+5], data['norm_AT'][76:76+5]]
test_data = [data['X'][76+5:], data['norm_A'][76+5:], data['norm_AT'][76+5:]]

In [18]:
reg = regressor(train_data, val_acc[:76])

In [19]:
y_pred = reg.predict(train_data)
y_true = val_acc[:76]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.8397596019859958 0.43530335973545037 0.3395288666076932


In [20]:
np.save('outputs/cifar100_y_true_train.npy', y_true)
np.save('outputs/cifar100_y_pred_train.npy', y_pred)
np.save('outputs/cifar100_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/cifar100_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/cifar100_r2_train.npy', r2_score(y_true, y_pred))

In [21]:
y_pred = reg.predict(valid_data)
y_true = val_acc[76:76+5]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

1.4579899197781454 0.0 -0.46229457615062186


In [22]:
np.save('outputs/cifar100_y_true_val.npy', y_true)
np.save('outputs/cifar100_y_pred_val.npy', y_pred)
np.save('outputs/cifar100_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/cifar100_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/cifar100_r2_val.npy', r2_score(y_true, y_pred))

In [23]:
y_pred = reg.predict(test_data)
y_true = val_acc[76+5:]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.2741968098290307 0.39999999999999997 0.28247827810860293


In [24]:
np.save('outputs/cifar100_y_true_test.npy', y_true)
np.save('outputs/cifar100_y_pred_test.npy', y_pred)
np.save('outputs/cifar100_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/cifar100_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/cifar100_r2_test.npy', r2_score(y_true, y_pred))

Caltech-101

In [25]:
models, val_acc = [], []

model_acc = np.load(f'outputs/model_acc_caltech.npy', allow_pickle=True)
for m in model_acc:
    models.append(m['arch'])
    val_acc.append(m['val_acc'] * 100.)

In [26]:
data = preprocess_proxylessnas(models)
val_acc = np.array(val_acc)
train_data = [data['X'][:208], data['norm_A'][:208], data['norm_AT'][:208]]
valid_data = [data['X'][208:208+40], data['norm_A'][208:208+40], data['norm_AT'][208:208+40]]
test_data = [data['X'][208+40:], data['norm_A'][208+40:], data['norm_AT'][208+40:]]

In [27]:
reg = regressor(train_data, val_acc[:208])

In [28]:
y_pred = reg.predict(train_data)
y_true = val_acc[:208]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

6.959254344589159 0.4041692228296266 0.37439728278496354


In [29]:
np.save('outputs/caltech_y_true_train.npy', y_true)
np.save('outputs/caltech_y_pred_train.npy', y_pred)
np.save('outputs/caltech_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/caltech_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/caltech_r2_train.npy', r2_score(y_true, y_pred))

In [30]:
y_pred = reg.predict(valid_data)
y_true = val_acc[208:208+40]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

9.021863658309812 0.22706868045228576 0.030166659813194552


In [31]:
np.save('outputs/caltech_y_true_val.npy', y_true)
np.save('outputs/caltech_y_pred_val.npy', y_pred)
np.save('outputs/caltech_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/caltech_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/caltech_r2_val.npy', r2_score(y_true, y_pred))

In [32]:
y_pred = reg.predict(test_data)
y_true = val_acc[208+40:]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

9.51458586000109 0.1862559655186504 -0.03665811883826575


In [33]:
np.save('outputs/caltech_y_true_test.npy', y_true)
np.save('outputs/caltech_y_pred_test.npy', y_pred)
np.save('outputs/caltech_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/caltech_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/caltech_r2_test.npy', r2_score(y_true, y_pred))

Oxford IIIT PET

In [34]:
models, val_acc = [], []

model_acc = np.load(f'outputs/model_acc_oxford_pet.npy', allow_pickle=True)
for m in model_acc:
    models.append(m['arch'])
    val_acc.append(m['val_acc'] * 100.)

In [35]:
data = preprocess_proxylessnas(models)
val_acc = np.array(val_acc)
train_data = [data['X'][:212], data['norm_A'][:212], data['norm_AT'][:212]]
valid_data = [data['X'][212:212+10], data['norm_A'][212:212+10], data['norm_AT'][212:212+10]]
test_data = [data['X'][212+10:], data['norm_A'][212+10:], data['norm_AT'][212+10:]]

In [36]:
reg = regressor(train_data, val_acc[:212], is_shift=False)

In [37]:
y_pred = reg.predict(train_data)
y_true = val_acc[:212]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.8656793516167571 0.2826455331616989 0.17840147231556114


In [38]:
np.save('outputs/oxford_y_true_train.npy', y_true)
np.save('outputs/oxford_y_pred_train.npy', y_pred)
np.save('outputs/oxford_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/oxford_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/oxford_r2_train.npy', r2_score(y_true, y_pred))

In [39]:
y_pred = reg.predict(valid_data)
y_true = val_acc[212:212+10]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.8102647996119685 0.22473328748774735 0.17869906043300154


In [40]:
np.save('outputs/oxford_y_true_val.npy', y_true)
np.save('outputs/oxford_y_pred_val.npy', y_pred)
np.save('outputs/oxford_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/oxford_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/oxford_r2_val.npy', r2_score(y_true, y_pred))

In [41]:
y_pred = reg.predict(test_data)
y_true = val_acc[212+10:]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.7684892989426564 -0.022733144649015782 -0.3563133691226925


In [42]:
np.save('outputs/oxford_y_true_test.npy', y_true)
np.save('outputs/oxford_y_pred_test.npy', y_pred)
np.save('outputs/oxford_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/oxford_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/oxford_r2_test.npy', r2_score(y_true, y_pred))

### NAS-Bench-NLP

In [43]:
N = 172

In [44]:
search_set, env = load_nasbench_nlp()

In [45]:
def get_N_nlp_samples(N):
    env.reset()
    
    models = []
    stats = []

    for _ in range(N):
        while True:
            idx = np.random.choice(len(search_set), 1, replace=False)[0]
            model = search_set[idx]
            
            if model not in models:
                env.simulated_train(model, 50)
                
                if 'OK' == env.get_model_status(model):   
                    models.append(model)
                    stats.append(env.get_model_stats(model, 49))
                    break
                
    return preprocess_nasbench_nlp(models, stats)

In [46]:
models = get_N_nlp_samples(119+10000)

In [47]:
reg = regressor([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]], models['val_loss'][:79], mode='loss')

In [48]:
y_pred = reg.predict([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]])
y_true = models['val_loss'][:79]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.32234077903422514 0.25478740668614086 0.2524043080192905


In [49]:
np.save('outputs/nlp_y_true_train.npy', y_true)
np.save('outputs/nlp_y_pred_train.npy', y_pred)
np.save('outputs/nlp_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nlp_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nlp_r2_train.npy', r2_score(y_true, y_pred))

In [50]:
y_pred = reg.predict([models['X'][79:119], models['norm_A'][79:119], models['norm_AT'][79:119]])
y_true = models['val_loss'][79:119]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.2678292393893943 0.23846153846153847 0.07886005582047706


In [51]:
np.save('outputs/nlp_y_true_val.npy', y_true)
np.save('outputs/nlp_y_pred_val.npy', y_pred)
np.save('outputs/nlp_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nlp_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nlp_r2_val.npy', r2_score(y_true, y_pred))

In [52]:
y_pred = reg.predict([models['X'][119:], models['norm_A'][119:], models['norm_AT'][119:]])
y_true = models['val_loss'][119:]

In [53]:
min_idx = np.argmin(y_pred.ravel())
max_idx = np.argmax(y_pred.ravel())

In [54]:
y_idx = np.where(y_pred > y_pred[min_idx])[0]

In [55]:
y_pred_10 = np.vstack([y_pred[min_idx], y_pred[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_pred[max_idx]]).ravel()
y_true_10 = np.hstack([y_true[min_idx], y_true[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_true[max_idx]]).ravel()
print(mean_squared_error(y_true_10, y_pred_10), tau(y_true_10, y_pred_10).correlation, r2_score(y_true_10, y_pred_10))

0.48322699770890426 0.06666666666666667 -0.07946753853717325


In [56]:
np.save('outputs/nlp_y_true_test.npy', y_true)
np.save('outputs/nlp_y_pred_test.npy', y_pred)
np.save('outputs/nlp_y_true_test_10.npy', y_true_10)
np.save('outputs/nlp_y_pred_test_10.npy', y_pred_10)
np.save('outputs/nlp_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nlp_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nlp_r2_test.npy', r2_score(y_true, y_pred))